In [14]:
import numpy as np
import pandas as pd
import os

os.environ["KERAS_BACKEND"] = "jax"

# Note that Keras should only be imported after the backend
# has been configured. The backend cannot be changed once the
# package is imported.
import keras
import tensorflow as tf

In [9]:
from google.colab import drive
drive.mount('/content/drive')

from os import getcwd
from os.path import join
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content'

In [29]:
# Load the data and split it between train and test sets
image_dir = join("drive", "MyDrive", "Colab Notebooks", "mel-spectograms")
ds = keras.preprocessing.image_dataset_from_directory(
    image_dir,
    image_size=(128,128),  # resize if needed
    batch_size=32,
    shuffle=True
)

Found 1166 files belonging to 4 classes.


In [41]:
# Split into train/test
train_size = int(0.8 * len(ds))
ds_train = ds.take(train_size)
ds_test = ds.skip(train_size)

def preprocess(x, y):
  x = tf.cast(x, tf.float32) / 255.0
  # If grayscale (1 channel), duplicate to 3 channels
  if x.shape[-1] == 1:
    x = tf.image.grayscale_to_rgb(x)
  return x, y

ds_train = ds_train.map(preprocess)
ds_test = ds_test.map(preprocess)

# Optionally, create a validation dataset from training data
val_size = 0.15
total_batches = sum(1 for _ in ds_train)
val_batches = int(total_batches * val_size)

ds_val = ds_train.take(val_batches)
ds_train_final = ds_train.skip(val_batches)

In [42]:
# Model parameters
num_classes = 4

base_model = keras.applications.EfficientNetB0(
  include_top=False,
  weights='imagenet',
  input_shape=(128, 128, 3)
)
base_model.trainable = False # Freeze base for transfer learning

inputs = keras.Input(shape=(128, 128, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(num_classes, activation='softmax')(x)

model = keras.Model(inputs, outputs)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [43]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 4, 4, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         5,124 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,054,695 (15.47 MB)

 Trainable params: 5,124 (20.02 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [44]:
model.compile(
  optimizer=keras.optimizers.Adam(learning_rate=1e-3),
  loss=keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy']
)

In [45]:
callbacks = [
  keras.callbacks.ModelCheckpoint(filepath="efficientnet_epoch_{epoch}.keras"),
  keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
]

epochs = 20

history = model.fit(
  ds_train_final,
  validation_data=ds_val,
  epochs=epochs,
  callbacks=callbacks
)

Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.2297 - loss: 1.4295 - val_accuracy: 0.2969 - val_loss: 1.3917
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 128ms/step - accuracy: 0.2061 - loss: 1.4435 - val_accuracy: 0.2266 - val_loss: 1.3858
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 126ms/step - accuracy: 0.3075 - loss: 1.3944 - val_accuracy: 0.2656 - val_loss: 1.3995
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 124ms/step - accuracy: 0.2665 - loss: 1.4378 - val_accuracy: 0.2422 - val_loss: 1.3888
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 123ms/step - accuracy: 0.2804 - loss: 1.4077 - val_accuracy: 0.2656 - val_loss: 1.3850
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 154ms/step - accuracy: 0.2306 - loss: 1.4275 - val_accuracy: 0.2188 - val_loss: 1.3879
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 0.2438 - loss: 1.4296 - val_accuracy: 0.2344 - val_loss: 1.3851
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 127ms/step - accuracy: 0.2468 - loss: 1.4258 - val_accuracy: 0.2

In [46]:
score = model.evaluate(ds_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 1.391090750694275
Test accuracy: 0.26050421595573425


In [48]:
# Unfreeze top layers for fine-tuning
base_model.trainable = True
fine_tune_at = len(base_model.layers) - 50 # unfreeze last 50 layers

for layer in base_model.layers:
  layer.trainable = False

In [49]:
# Compile the model with a lower learning rate
model.compile(
  optimizer=keras.optimizers.Adam(learning_rate=1e-5), # lower LR for fine-tuning
  loss=keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy']
)

In [50]:
# Fine-tune
fine_tune_epochs = 10
total_epochs = 20 + fine_tune_epochs # initial + fine-tune

history_fine = model.fit(
  ds_train_final,
  validation_data=ds_val,
  epochs=total_epochs,
  initial_epoch=history.epoch[-1], # continue from previous training
  callbacks=callbacks
)

Epoch 11/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 711ms/step - accuracy: 0.2338 - loss: 1.4258 - val_accuracy: 0.2734 - val_loss: 1.3845
Epoch 12/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 94ms/step - accuracy: 0.2836 - loss: 1.3966 - val_accuracy: 0.2109 - val_loss: 1.3936
Epoch 13/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 0.2503 - loss: 1.4243 - val_accuracy: 0.2344 - val_loss: 1.3904
Epoch 14/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 139ms/step - accuracy: 0.2564 - loss: 1.4002 - val_accuracy: 0.2734 - val_loss: 1.3878


In [51]:
score = model.evaluate(ds_test, verbose=0)
print("Test loss after fine-tuning:", score[0])
print("Test accuracy after fine-tuning:", score[1])

Test loss after fine-tuning: 1.3770619630813599
Test accuracy after fine-tuning: 0.31092438101768494


In [52]:
model.save(os.path.join("models", "efficientnet.keras"))